In [2]:
!pip install datasets
!pip install transformers==4.45.2 sentence-transformers==3.1.1
!pip install rouge_score
!pip install evaluate
!pip install hazm
!pip install rouge

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [1]:
import datasets
import pandas as pd
from IPython.display import display, HTML
from datasets import ClassLabel
from transformers import BertTokenizerFast
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments,AutoTokenizer
from transformers import EncoderDecoderModel
from transformers import BertTokenizer
from rouge import Rouge
import numpy as np
from transformers import EncoderDecoderModel, BertModel, BertConfig
import torch.nn as nn
from hazm import *

In [1]:
!pip uninstall numpy==1.24.3 && pip install numpy==1.26.2

Found existing installation: numpy 1.24.3
Uninstalling numpy-1.24.3:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.11
    /usr/local/lib/python3.11/dist-packages/numpy-1.24.3.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libopenblas64_p-r0-15028c96.3.21.so
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? y
  Successfully uninstalled numpy-1.24.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 92.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
hazm 0.10.0 requires numpy==1.24.3, but you have numpy 1.26.2 whi

In [ ]:
import numpy
print(numpy.__version__)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
sample_size = 10000
encoder_max_length = 512
decoder_max_length = 128
#train_data = train_data.select(range(1000))
# batch_size = 16
batch_size = 4

In [4]:
train_data = datasets.load_dataset(
    "pn_summary", split="train", download_mode="force_redownload"
)
train_data = train_data.select(range(sample_size))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

pn_summary.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

pn_summary.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

The repository for pn_summary contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/pn_summary.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


pn_summary.zip:   0%|          | 0.00/89.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/82022 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5592 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5593 [00:00<?, ? examples/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

In [6]:
normalizer = Normalizer()

def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    for i in range(len(batch['article'])):
      batch['article'][i]=normalizer.normalize(batch['article'][i])
      batch['summary'][i]=normalizer.normalize(batch['summary'][i])

    inputs = tokenizer(
        batch["article"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
    )
    outputs = tokenizer(
        batch["summary"],
        padding="max_length",
        truncation=True,
        max_length=decoder_max_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()

    # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`.
    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [0 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [7]:
metric = Rouge()
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions

    labels = np.where(labels != 0, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Compute ROUGE scores
    result = metric.get_scores(decoded_preds, decoded_labels,avg=True)
    print(result)
    with open("/content/drive/MyDrive/result_bert2bert_decoder.txt","a") as handler:
        handler.write(f'rouge-1 - recall: {result["rouge-1"]["r"]} - precision: {result["rouge-1"]["p"]} - fscore: {result["rouge-1"]["f"]}\n')
        handler.write(f'rouge-2 - recall: {result["rouge-2"]["r"]} - precision: {result["rouge-2"]["p"]} - fscore: {result["rouge-2"]["f"]}\n')
        handler.write(f'rouge-l - recall: {result["rouge-l"]["r"]} - precision: {result["rouge-l"]["p"]} - fscore: {result["rouge-l"]["f"]}\n\n')
    # Extract F-measure for each ROUGE score
    rouge_result = {
        "rouge1": result["rouge-1"]["f"],
        "rouge2": result["rouge-2"]["f"],
        "rougeL": result["rouge-l"]["f"],
    }

    return rouge_result

In [8]:
train_data = train_data.map(
        process_data_to_model_inputs,
        batched=True,
        batch_size=batch_size,
        remove_columns=['id', 'title', 'article', 'summary', 'category', 'categories', 'network', 'link'],
    )

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [9]:
train_data.set_format(
        type="torch",
        columns=["input_ids", "attention_mask", "labels"],
    )

In [10]:
val_data = datasets.load_dataset(
        "pn_summary", split="validation[:30%]"
    )

In [11]:
val_data = val_data.map(
        process_data_to_model_inputs,
        batched=True,
        batch_size=batch_size,
        remove_columns=['id', 'title', 'article', 'summary', 'category', 'categories', 'network', 'link'],
    )

Map:   0%|          | 0/1678 [00:00<?, ? examples/s]

In [12]:
val_data.set_format(
        type="torch",
        columns=["input_ids", "attention_mask", "labels"],
    )

In [13]:
class PostDecoderCustomBert(EncoderDecoderModel):
    def __init__(self, config=None, encoder=None, decoder=None):
        super().__init__(config=config, encoder=encoder, decoder=decoder)

        # Add custom layer after decoder output
        decoder_hidden_size = self.decoder.config.hidden_size
        vocab_size = self.decoder.config.vocab_size

        # Custom layer for summarization refinement
        self.post_decoder_layer = nn.Sequential(
            nn.Linear(decoder_hidden_size, decoder_hidden_size * 2),
            nn.GELU(),  # Better for text generation than ReLU
            nn.Dropout(0.1),
            nn.Linear(decoder_hidden_size * 2, vocab_size),
            nn.LogSoftmax(dim=-1)  # For better numerical stability
        )

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        decoder_input_ids=None,
        decoder_attention_mask=None,
        labels=None,
        **kwargs
    ):
        # Standard encoder-decoder forward pass
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=None,  # We'll handle loss computation ourselves
            output_hidden_states=True,
            return_dict=True,
            **kwargs
        )

        # Get decoder's last hidden states
        decoder_hidden_states = outputs.decoder_hidden_states[-1]

        # Pass through custom layer
        logits = self.post_decoder_layer(decoder_hidden_states)

        # Compute loss if labels provided
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
            loss = loss_fct(logits.view(-1, self.decoder.config.vocab_size),
                           labels.view(-1))

        return {
            'loss': loss,
            'logits': logits,
            'encoder_last_hidden_state': outputs.encoder_last_hidden_state,
            'decoder_hidden_states': outputs.decoder_hidden_states
        }

In [14]:
encoder_config = BertConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
decoder_config = BertConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

# Decoder settings
decoder_config.is_decoder = True
decoder_config.add_cross_attention = True

# Create model
bert2bert = PostDecoderCustomBert(
    encoder=BertModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased", config=encoder_config),
    decoder=BertModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased", config=decoder_config)
)
bert2bert.save_pretrained("bert2bert")
bert2bert = EncoderDecoderModel.from_pretrained("bert2bert")

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder.layer.1.crossattention.output.dense.weight', 'bert.encoder.layer.1.crossattention.self.ke

In [15]:
bert2bert.config.decoder_start_token_id = tokenizer.cls_token_id
bert2bert.config.eos_token_id = tokenizer.sep_token_id
bert2bert.config.pad_token_id = tokenizer.pad_token_id
bert2bert.config.vocab_size = bert2bert.config.encoder.vocab_size
bert2bert.config.max_length = 128
bert2bert.config.min_length = 0
bert2bert.config.no_repeat_ngram_size = 2
bert2bert.config.early_stopping = True
bert2bert.config.length_penalty = 2.0
bert2bert.config.num_beams = 3

In [16]:
training_args = Seq2SeqTrainingArguments(
        predict_with_generate=True,
        evaluation_strategy="steps",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        fp16=True,
        output_dir="/content/drive/MyDrive/bert2bert_model",
        logging_steps=2,
        save_steps=2000,
        eval_steps=2000,
        num_train_epochs=5,
        report_to="none",
        warmup_steps=1000,
    )

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(
        model=bert2bert,
        tokenizer=tokenizer,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_data,
        eval_dataset=val_data,
    )
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:623: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel
2000,1.359000,1.433949,0.259162,0.056571,0.205537
4000,1.384600,1.283549,0.296544,0.084487,0.241088
6000,1.066500,1.211203,0.321978,0.112461,0.265400
8000,0.766200,1.165968,0.334786,0.122915,0.277590
10000,0.960400,1.137187,0.346948,0.129589,0.286547
12000,0.809300,1.131829,0.352817,0.136378,0.291084


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1338: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


{'rouge-1': {'r': 0.274320440125754, 'p': 0.2609293295091681, 'f': 0.25916196800230573}, 'rouge-2': {'r': 0.07303618972582698, 'p': 0.04980497634954261, 'f': 0.056570960870620586}, 'rouge-l': {'r': 0.21851671382475035, 'p': 0.20618658540407317, 'f': 0.20553658467494892}}


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'early_stopping': True, 'num_beams': 3, 'length_penalty': 2.0, 'no_repeat_ngram_size': 2}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:623: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to 

...

{'rouge-1': {'r': 0.35573292081132024, 'p': 0.33212616256321326, 'f': 0.33478566647714936}, 'rouge-2': {'r': 0.14144798159280933, 'p': 0.11630266726307778, 'f': 0.12291477234699993}, 'rouge-l': {'r': 0.2957149727529516, 'p': 0.2748812482474735, 'f': 0.27759027610826065}}


/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:623: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'rouge-1': {'r': 0.37659259057458455, 'p': 0.3370387418976425, 'f': 0.34694829599579413}, 'rouge-2': {'r': 0.1539807377352874, 'p': 0.11940198362959777, 'f': 0.1295893756377216}, 'rouge-l': {'r': 0.31186187543259775, 'p': 0.277696952461338, 'f': 0.28654700212394946}}


/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:623: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'rouge-1': {'r': 0.3750892912987729, 'p': 0.3493578189205872, 'f': 0.3528174099223035}, 'rouge-2': {'r': 0.15612054397270345, 'p': 0.12926881138479934, 'f': 0.13637848814620407}, 'rouge-l': {'r': 0.3104750607438949, 'p': 0.287425459933337, 'f': 0.2910836415002403}}
